In [7]:
# Imports
from ypstruct import structure
import numpy as np

#### Replicating the example evaluation of a candidate solution found in as found https://link.springer.com/chapter/10.1007/978-3-319-54157-0_4

Setting up the problem example

In [8]:
item_values = np.array([30, 34, 40, 25])
item_weights = np.array([25, 30, 40, 21])

item_to_city_mapping = np.array(
    [
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
    ]
)

distance_matrix = np.array(
    [
    [0, 4, 9, 3],
    [4, 0, 5, 5],
    [9, 5, 0, 4],
    [3, 5, 4, 0]
    ]
)

vmax = 1.0
vmin = 0.1
Q = 80

Cost function for the knapsack, as given by:
$$
  \sum^m_{j = 1} z_j * b_j
$$
where $z_j$ is the a binary vector where each index corresponds to an item and is 0 if the item is not picked and 1 if it is and $b_j$ is the values vector.

In [9]:
def knapsack_cost(candidate_solution, item_values):
    return np.sum(candidate_solution.knapsack * item_values)

Cost function for the tour, taking into account velocity, as given by:
$$
    f(\pi, z) = \frac{d_{\pi_1, \pi_{n}}}{v(w(\pi_{n}))} + \sum^{n-1}_{i = 1} \frac{d_{\pi_i, \pi_{i+1}}}{v(w(\pi_{i}))}
$$ 
where 
$$
    w(\pi_i) = \sum^{i}_{k=1}\sum^{m}_{j=1} z_j * b_j * a_{j, \pi_i}
$$ 

and 
$$
    v(q) = v_{max} - \frac{q}{Q} * (v_{max} - v_{min})
$$ 

In [10]:
def tour_cost(candidate_solution, distance_matrix, item_to_city_mapping, item_weights):
    tour = candidate_solution.tour
    
    # calculate the weight picked up at each city
    weight_at_city_i = []
    for j in range(0, len(tour)):
        item_in_city = np.array([col[tour[j]-1] for col in item_to_city_mapping])
        weight_at_city_i.append(sum(item_in_city * candidate_solution.knapsack * item_weights))
    # calculate the total weight that the thief has when leaving each city using the cumulative sum 
    weight_at_city_i = np.array(np.cumsum(weight_at_city_i))

    # calculate the velocity of the thief when leaving each city
    velocity_at_city_i = vmax - weight_at_city_i / Q * (vmax - vmin)

    # retrieve the distance between the cities 
    distance_between_cities = []
    for i in range(0, len(tour)-1):
        distance_between_cities.append(distance_matrix[tour[i]-1][tour[i+1]-1])
    distance_between_cities.append(distance_matrix[0][-1])
    distance_matrix = np.array(distance_between_cities)
    
    # return the total completion time
    return np.sum(distance_between_cities / velocity_at_city_i)

Setting up the candidate solution and evaluating it using the previously defined function

In [11]:
candidate_solution = structure()

candidate_solution.tour = np.array([1, 3, 2, 4]) # 4 cities
candidate_solution.knapsack = np.array([0, 1, 0, 1]) # 4 items
candidate_solution.tour_cost = tour_cost(candidate_solution, distance_matrix, item_to_city_mapping, item_weights)
candidate_solution.knapsack_cost = knapsack_cost(candidate_solution, item_values)

print(candidate_solution)
print(candidate_solution.tour_cost)
print(candidate_solution.knapsack_cost)

struct({'tour': array([1, 3, 2, 4]), 'knapsack': array([0, 1, 0, 1]), 'tour_cost': 28.585292978476183, 'knapsack_cost': 59})
28.585292978476183
59
